# Add sentence reference to each sentence in every story for each word in common words

Ref: storynumber_sentencenumber

In [4]:
import pandas as pd
import os
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\haafla\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\haafla\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [12]:
# Open common_words.csv
common_words = pd.read_csv('./data/common_words.csv', delimiter=';')
# nltk.download('popular')

# Open all stories inside the stories folder
for file in os.listdir('./stories'):
  story_num = file.split('_')[0]
  with open('./stories/' + file, 'r') as f:
    story = f.read()
    
  it_story = story.split(';;;')[0]
  # Split the story into sentences
  sentences = sent_tokenize(it_story)
  
  # Find sentence refernces for each common word
  common_words['Sentence references'] = ''
  for index, row in common_words.iterrows():
    word = row['Word']
    sentence_references = []
    for i, sentence in enumerate(sentences):
      if word in sentence:
        sentence_references.append(f'{story_num}_{i}')
    common_words.at[index, 'Sentence references'] = sentence_references
    
  # Save the common_words dataframe
  common_words.to_csv('./data/common_words.csv', sep=';', index=False)